# 0. Librerías


In [2]:
!pip install plotly_express
import plotly_express as px
!pip install plotly --upgrade
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.metrics import mean_absolute_error
import seaborn as sns


In [26]:

df = pd.read_csv('datos_ejercicio_ventas.csv', sep = ",")
print(df)
df_actual = df[df['SCENARIO'] == 'actual']

             COUNTRY            SUBBRAND  YEAR  MONTH     SCENARIO FORECAST  \
0           Portugal         Lipton (L3)  2023     12  AI_forecast  AI_P02F   
1      Great Britain         Lipton (L3)  2023     12  AI_forecast  AI_P10F   
2              Spain      Pepsi Max (L3)  2023     12  AI_forecast  AI_P09F   
3      Great Britain            7up (L3)  2024     12  AI_forecast  AI_P10F   
4            Hungary         Lipton (L3)  2023      9  AI_forecast  AI_P03F   
...              ...                 ...   ...    ...          ...      ...   
18661  Great Britain  Pepsi Regular (L3)  2024      2  AI_forecast  AI_P10F   
18662        Hungary  Pepsi Regular (L3)  2024      7  AI_forecast  AI_P07F   
18663         Norway            7up (L3)  2024      1  AI_forecast  AI_P05F   
18664       Portugal         Lipton (L3)  2024      3  AI_forecast  AI_P02F   
18665        Hungary   Mountain Dew (L3)  2024      8  AI_forecast  AI_P09F   

       FORECAST_YEAR        AMOUNT  
0             

# 1. Analizar cómo se distribuyen las ventas realizadas en:


## 1.1. Cada país

In [4]:
fig_country = px.histogram(
    df_actual,
    x='COUNTRY',
    y='AMOUNT',
    color='COUNTRY',
    title='Distribución de Ventas por País',
    labels={'AMOUNT': 'Monto de Ventas', 'COUNTRY': 'País'}
)
fig_country.show()

## 1.2. Cada mes y año


In [5]:
# Creamos la fecha completa
df_actual['DATE'] = pd.to_datetime(df_actual[['YEAR', 'MONTH']].assign(DAY=1))

# Agrupar y sumar las ventas por fecha (por mes)
sales_by_date = df_actual.groupby('DATE')['AMOUNT'].sum().reset_index()

# Crear el gráfico con plotly.express
fig = px.line(sales_by_date, x='DATE', y='AMOUNT',
              title='Ventas realizadas',
              labels={'DATE': 'Fecha', 'AMOUNT': 'Total de Ventas'},
              markers=True)

# Ajustes adicionales
fig.update_xaxes(title_text='Fecha', tickangle=-45)
fig.update_yaxes(title_text='Total de Ventas')
fig.update_layout(title_font_size=14, xaxis_title_font_size=10, yaxis_title_font_size=10)

# Mostrar el gráfico
fig.show()

<ipython-input-5-744d7fc0a928>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## 1.3. Cada marca

In [6]:
fig_brand = px.histogram(
    df_actual,
    x='SUBBRAND',
    y='AMOUNT',
    color='SUBBRAND',
    title='Distribución de Ventas por Marca',
    labels={'AMOUNT': 'Monto de Ventas', 'SUBBRAND': 'Marca'}
)
fig_brand.show()

# 2. Cual es la tendencia y estacionalidad de:


## 2.1. Todas las ventas del país con menos ventas


In [7]:
# Filtrar solo los registros donde SCENARIO es "actual"
df_actual = df[df['SCENARIO'] == 'actual'].copy()  # Usar .copy() para evitar el warning

# Crear la fecha completa
df_actual['DATE'] = pd.to_datetime(df_actual[['YEAR', 'MONTH']].assign(DAY=1))

# Identificar el país con menos ventas en el conjunto filtrado
total_ventas_por_pais = df_actual.groupby('COUNTRY')['AMOUNT'].sum().sort_values()
print(total_ventas_por_pais)
pais_menor_ventas = total_ventas_por_pais.index[0]  # país con menos ventas

# Filtrar los datos del país con menos ventas
ventas_pais = df_actual[df_actual['COUNTRY'] == pais_menor_ventas]

# Ordenar los datos del país con menos ventas por fecha
ventas_pais = ventas_pais.sort_values(by='DATE')  # Ahora es un DataFrame

# Agrupar por fecha para el análisis de tendencia y estacionalidad
ventas_pais_grouped = ventas_pais.groupby('DATE')['AMOUNT'].sum()

# Calcular la media móvil (ajustar el periodo según sea necesario)
ventas_pais_media = ventas_pais_grouped.rolling(window=7, min_periods=1).mean().reset_index()
ventas_pais_grouped = ventas_pais_grouped.reset_index()

# Crear el gráfico con Plotly Express
fig = px.line(ventas_pais_grouped, x='DATE', y='AMOUNT', title=f'Tendencia y Estacionalidad - Ventas en {pais_menor_ventas}',
              labels={'DATE': 'Fecha', 'AMOUNT': 'Monto de Ventas'})
fig.add_scatter(x=ventas_pais_media['DATE'], y=ventas_pais_media['AMOUNT'], mode='lines', name='Media Móvil', line=dict(color='orange', dash='dash'))

# Mostrar el gráfico
fig.update_xaxes(title_text='Fecha')
fig.update_yaxes(title_text='Monto de Ventas')
fig.update_layout(legend_title_text='Tipo de Serie', xaxis_tickangle=-45)
fig.show()

COUNTRY
Spain            8.131266e+06
Portugal         3.488807e+07
Czech            3.535164e+07
Hungary          4.153991e+07
Italy            4.345404e+07
Norway           5.121406e+07
Denmark          5.659668e+07
Netherlands      6.395943e+07
Great Britain    3.347786e+08
Name: AMOUNT, dtype: float64


Podemos ver que el país con menos ventas es España.
La línea azul (ventas) muestra picos en las ventas alrededor de febrero y septiembre de 2023, con un máximo en julio de 2023. Después, las ventas disminuyen hasta alcanzar uno de sus puntos más bajos en febrero de 2024 y a partir de entonces las ventas comienzan a recuperarse.
La línea amarilla (media móvil), representa la tendencia suavizada de las ventas a lo largo del tiempo. Se ve un aumento gradual hasta mediados de 2023, que decrece hasta principios de 2024. Después vuelve a subir, indicando una mejoría en las ventas.


In [8]:
# Filtrar solo los registros donde SCENARIO es "actual"
df_actual = df[df['SCENARIO'] == 'actual'].copy()

# Identificar la marca con más ventas
total_ventas_por_marca = df_actual.groupby('SUBBRAND')['AMOUNT'].sum().sort_values(ascending=False)
marca_mayor_ventas = total_ventas_por_marca.index[0]

# Agrupar las ventas por año y mes
ventas_marca = df_actual[df_actual['SUBBRAND'] == marca_mayor_ventas].groupby(['YEAR', 'MONTH'])['AMOUNT'].sum()

# Calcular la media móvil (ajustar el periodo según sea necesario)
ventas_marca_media = ventas_marca.rolling(window=3, min_periods=1).mean()

# Convertir el índice a formato de fecha para plotly
ventas_marca.index = pd.to_datetime(ventas_marca.index.map(lambda x: f"{x[0]}-{x[1]:02d}"))
ventas_marca_media.index = ventas_marca.index  # Aseguramos que ambos índices coincidan

# Crear un DataFrame combinado para las ventas y la media móvil
df_trend = pd.DataFrame({
    'Fecha': ventas_marca.index,
    'Ventas': ventas_marca.values,
    'Media Móvil': ventas_marca_media.values
})

# Crear el gráfico con plotly.express
fig = px.line(df_trend, x='Fecha', y=['Ventas', 'Media Móvil'],
              labels={'value': 'Monto de Ventas', 'variable': 'Serie'},
              title=f'Tendencia y Estacionalidad - Ventas de {marca_mayor_ventas}')

# Ajustes adicionales
fig.update_layout(legend_title_text='Serie')
fig.update_traces(marker=dict(size=5))
fig.show()

En este caso, hemos analizado el subproducto con más ventas a lo largo de 2023 y lo que llevamos de 2024, y hemos visto que Pepsi Max(L3) es el producto mayor vendido a lo largo de los países de la base de datos.
Este modelo cuenta con una línea azul, que representa las ventas reales, y una amarilla que representa la media móvil.

Podemos ver que la línea azul, a pesar de sus constantes picos, positivos y negativos, se mentiene en un monto medio de ventas, alcanzando su mínimo en diciembre de 2023 y alcanzando su máximo en abril de 2024.
Así mismo, la media móvil nos muestra un suave crecimiento, levemente afectado en el primer cuatrimestre de 2024, aunque rápidamente recupera.

#3. Cuáles son las predicciones hechas en España y cómo de buenas son.


In [25]:
import plotly.graph_objects as go
import pandas as pd

# Filtrar los datos para España
df_spain = df[df['COUNTRY'] == 'Spain'].copy()

# Crear DataFrames separados para las ventas actuales y predichas
df_actual = df_spain[df_spain['SCENARIO'] == 'actual'].copy()
df_forecast = df_spain[df_spain['SCENARIO'] == 'AI_forecast'].copy()

# Crear la columna de fecha
df_actual['DATE'] = pd.to_datetime(df_actual[['YEAR', 'MONTH']].assign(DAY=1))
df_forecast['DATE'] = pd.to_datetime(df_forecast[['YEAR', 'MONTH']].assign(DAY=1))

# Agrupar las ventas reales por mes y año
sales_actual = df_actual.groupby('DATE')['AMOUNT'].sum().reset_index()

# Crear el gráfico con plotly
fig = go.Figure()

# Añadir la línea de ventas reales
fig.add_trace(go.Scatter(
    x=sales_actual['DATE'],
    y=sales_actual['AMOUNT'],
    mode='lines+markers',
    name='Ventas Reales (Actual)',
    line=dict(color='blue')
))

# Añadir cada línea de predicción individual (AI_P02F, AI_P03F, ..., AI_P12F)
for forecast_type in df_forecast['FORECAST'].unique():
    forecast_data = df_forecast[df_forecast['FORECAST'] == forecast_type]
    sales_forecast = forecast_data.groupby('DATE')['AMOUNT'].sum().reset_index()

    fig.add_trace(go.Scatter(
        x=sales_forecast['DATE'],
        y=sales_forecast['AMOUNT'],
        mode='lines+markers',
        name=f'Predicción {forecast_type}',
        line=dict(dash='dash')  # Línea discontinua para diferenciar predicciones de las ventas reales
    ))

# Configuración del diseño del gráfico
fig.update_layout(
    title='Comparación de Ventas Reales vs. Predicciones Individuales (España)',
    xaxis_title='Fecha',
    yaxis_title='Cantidad Ventas',
    legend=dict(orientation="v", yanchor="bottom", y=1.02, xanchor="center", x=0.8),
    template="plotly_white"
)

fig.show()

Para analizar cómo de buenas han sido las predicciones en España (teniendo en cuenta que teníamos 100 datos actuales y 2057 datos predictivos), se ha decidido hacer la suma entre las observaciones (teniendo en cuenta mes y año) dependiendo del forecast. Además de haber previamente filtrado y separado los datos por COUNTRY (Spain) por FORECAST ('AI_P10F' 'AI_P09F' 'AI_P03F' 'AI_PF' 'AI_P11F' 'AI_P06F' 'AI_P05F' 'AI_P07F' 'AI_P12F' 'AI_P08F' 'AI_P04F' y none) y por SCENARIO (actual y AI_forecast) y centrándose en la fecha y el dinero ganado.
- La línea inferior azul, representa las ventas reales de cada producto.
- Las lineas discontinuas representan las ventas predichas de los productos.


Podemos ver, que algunas predicciones son, por lo general superiores a las reales, por lo que se puede decir que los modelos de predicción tienden a sobreestimar el volumen de ventas.
Aunque las predicciones siguen un patrón similar al de las ventas reales, con picos en los mismos periodos, los niveles absolutos de las predicciones son superiores.


In [10]:
import numpy as np
# Asegúrate de que ambos DataFrames tengan la misma longitud y estén alineados por fecha
merged_data = pd.merge(sales_actual, sales_forecast, on='DATE', suffixes=('_actual', '_forecast'))

# Calcular las métricas
mae = np.mean(np.abs(merged_data['AMOUNT_forecast'] - merged_data['AMOUNT_actual']))
mse = np.mean((merged_data['AMOUNT_forecast'] - merged_data['AMOUNT_actual']) ** 2)
rmse = np.sqrt(mse)
mape = np.mean(np.abs((merged_data['AMOUNT_forecast'] - merged_data['AMOUNT_actual']) / merged_data['AMOUNT_actual'])) * 100

# Mostrar resultados
print(f'Error Absoluto Medio (MAE): {mae}')
print(f'Error Cuadrático Medio (MSE): {mse}')
print(f'Raíz del Error Cuadrático Medio (RMSE): {rmse}')
print(f'Porcentaje de Error Absoluto Medio (MAPE): {mape}%')

Error Absoluto Medio (MAE): 7212337.919838761
Error Cuadrático Medio (MSE): 70528614810377.02
Raíz del Error Cuadrático Medio (RMSE): 8398131.62616406
Porcentaje de Error Absoluto Medio (MAPE): 1736.2834961067456%
